In [1]:
import numpy as np 
import pandas as pd 
import os 
#from pandas_datareader import data
from datetime import timedelta, date, datetime

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt

import data_modules as dm

In [2]:
# Get file paths and directories
main_WorkingDir=os.path.abspath(os.curdir)#current working Directory
yahoo_data = os.path.join(main_WorkingDir,"yahoo_prepared_df")
redd_dir = os.path.join(main_WorkingDir,"redditStock_data")
avg_Dir = os.path.join(redd_dir,"sentiment_avg")

In [7]:
os.chdir(yahoo_data)

symlist = ['AAPL', 'ADI', 'AMT', 'AMZN', 'BABA', 'COUP', 'CRM', 'ERIC', 'GOOGL', 'MELI', 'MSFT', 'NVDA', 'OKTA', 'PANW', 'PYPL', 'QCOM', 'SHOP', 'SQ', 'TMUS']

weekly_or_monthly = 'monthly'
append_df=[] #list of dataframes
#append_X=[]
append_Y=[]
for symbol in symlist:
  df = pd.read_csv(symbol+'.csv',index_col=0)
  
  Y=dm.prepare_data_Y_for_ML(df, weekly_or_monthly)
  append_Y.append(Y)
#   X=dm.prepare_data_X_for_ML(df, weekly_or_monthly)
#   X.drop(['Adj Close', 'Volume'], axis=1, inplace=True)
#   append_X.append(X)
  append_df.append(df)
df2=pd.concat(append_df)
# df2.shape

# Xall=pd.concat(append_X)
Yall=pd.concat(append_Y)
print(Yall)

#os.chdir(avg_Dir)
os.chdir(main_WorkingDir)
temp_df = pd.read_csv("allRedditStocks.csv")
print(temp_df)
Xall = pd.DataFrame(columns=['Date'])
start_col = False
for ind, row in temp_df.iterrows():
    if row['Unnamed: 0.1'] == '2020-01-06':
            start_col = True
    elif row['Unnamed: 0.1'] =='2021-01-29':
        start_col = False
    if start_col:
        if weekly_or_monthly == 'monthly':
            # df = pd.DataFrame({'Date': row['Unnamed: 0.1'], 'sentiment':row['mid term average']})
            # Xall = Xall.append(df)
            Xall = Xall.append(pd.DataFrame([row['Unnamed: 0.1']],columns=['Date']))
            #Xall = Xall.append([row['Unnamed: 0.1'], row['mid term average']])

Xall['Date'] = Xall['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
Xall = Xall.set_index('Date')
dates = pd.date_range(Xall.index[0], Xall.index[-1], freq='D')

Xall.reindex(dates)
print(Xall)





#df = pd.read_csv(symlist[0]+'_avg.csv')
# df['Unnamed: 0']

# Xall = pd.DataFrame()
# start_col = False
# for symbol in symlist:
#     df = pd.read_csv(symbol + '_avg.csv')
#     for ind, row in df.iterrows():
#         if row['Unnamed: 0'] == '2020-01-06':
#             start_col = True
#         elif row['Unnamed: 0'] =='2021-01-29':
#             start_col = False
#         if start_col:
#             if weekly_or_monthly == 'monthly':
#                 Xall = Xall.append(pd.DataFrame([row['Unnamed: 0'], row['mid term average']]))
# print(Xall)







Date
2020-07-28   -13.715456
2020-07-29   -14.946342
2020-07-30   -15.500573
2020-07-31   -24.167138
2020-08-03   -23.924262
                ...    
2021-01-25    -2.318847
2021-01-26    -2.031807
2021-01-27     7.000890
2021-01-28     4.046338
2021-01-29     6.067571
Name: Ground Truth, Length: 2451, dtype: float64
       Unnamed: 0 Unnamed: 0.1  Daily average  weekly average  \
0               0   2019-02-13      -0.111000             NaN   
1               1   2019-02-14       0.308382             NaN   
2               2   2019-02-15      -0.122157             NaN   
3               3   2019-02-16       0.381512             NaN   
4               4   2019-02-17       0.298000             NaN   
...           ...          ...            ...             ...   
13893         726   2021-02-08      -0.000483        0.159516   
13894         727   2021-02-09      -0.169367        0.080875   
13895         728   2021-02-10      -0.029840       -0.003706   
13896         729   2021-02-11  

ValueError: cannot reindex from a duplicate axis

In [4]:
# Train on all  data
# make sure dates are same for both
X_train,X_test,y_train,y_test = train_test_split(Xall,Yall,test_size=0.25,random_state=0)
out=dm.apply_MLP(X_train,X_test,y_train,y_test)
r2score_all=r2_score(out['Ground Truth'], out['Prediction'])
print(out)
print("The Score with ", (r2_score(out['Ground Truth'], out['Prediction'])))

ValueError: Found input variables with inconsistent numbers of samples: [13898, 2451]

In [ ]:
#Train individually 
r2score_list = []
for symbol in symlist:
  df=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/yahoo_prepared_df/{}.csv'.format(symbol), index_col=0)
  
  Y=prepare_data_Y_for_ML(df, weekly_or_monthly)
  X=prepare_data_X_for_ML(df, weekly_or_monthly)
  X.drop(['Adj Close', 'Volume'], axis=1, inplace=True)
  #X=X.drop(['Adj Close'],axis=1)
  X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.25,random_state=0)
  out=apply_MLP(X_train,X_test,y_train,y_test)
  r2score_list.append(r2_score(out['Ground Truth'], out['Prediction']))

In [ ]:
plt.figure(figsize=(12,5))
plt.xlabel('Dates')

ax1 = out["Prediction"].plot(color='blue', grid=True)
ax2 = out["Ground Truth"].plot(color='red', grid=True)

h1, l1 = ax1.get_legend_handles_labels()
h2, l2 = ax2.get_legend_handles_labels()

plt.legend(h1, l1, loc=2)
plt.show()

In [ ]:
symlist
r2score_list
df_r2 = pd.DataFrame({'r2':r2score_list}, index=symlist)
df_r2=df_r2.append(pd.DataFrame({'r2':r2score_all}, index=['All Stocks']))
df_r2.sort_values(by='r2',axis=0,inplace=True,ascending=True)
ax=df_r2.plot.barh()
ax.set_xlabel('R2 score')
ax.legend('')
mytitle='MLP regression on ' +weekly_or_monthly + ' basis'
mytitle
ax.set_title(mytitle)
# fig, ax = plt.subplots()
# y_pos=np.arange(len(symlist))
# ax.barh(y_pos, r2score_list)
# plt.show()